### ADC. MCP3008 in Java
Make sure you have compiled and archived the required resources:

From the `ADC` folder _on the Raspberry Pi_, make sure you have run
```
 ADC $ ../gradlew shadowJar
```
This will generate the resources to import later on.

Here is the wiring on the Raspberry Pi:

![Wiring](./RPi-MCP3008-Pot_bb.png)


Here is a way to add a `jar` to the classpath:

In [1]:
List<String> added = %jars ../../ADC/build/libs/ADC-1.0-all.jar

Archive has been created, we can import the required resources:

In [2]:
import analogdigitalconverter.mcp3008.MCPReader;
import com.pi4j.io.gpio.Pin;
import utils.PinUtil;
import utils.StringUtils;

Identify the Channel to read. We can have up to 8 on an MCP3008.

In [3]:
// Between 0 and 7, 8 channels on the MCP3008
private static int adcChannel = MCP3008Reader.MCP3008_input_channels.CH0.ch(); 

private final static boolean DEBUG = true;

Identify the Pins used on the Raspberry pi to communicate with the `MCP3008`, using the SPI protocol.

In [4]:
// Default pins
Pin miso = PinUtil.GPIOPin.GPIO_4.pin();
Pin mosi = PinUtil.GPIOPin.GPIO_5.pin();
Pin clk  = PinUtil.GPIOPin.GPIO_1.pin();
Pin cs   = PinUtil.GPIOPin.GPIO_6.pin();

Let's print out the wiring of the Raspberry Pi and the `MCP3008`, to see if it matches the wiring above:

In [5]:
System.out.println(String.format("Reading MCP3008 on channel %d", adcChannel));
System.out.println(
            " Wiring of the MCP3008-SPI (without power supply):\n" +
            " +---------++-------------------------------------------------+\n" +
            " | MCP3008 || Raspberry Pi                                    |\n" +
            " +---------++------+--------------+------+---------+----------+\n" +
            " |         || Pin# | Name         | Role | GPIO    | wiringPI |\n" +
            " |         ||      |              |      | /BCM    | /PI4J    |\n" +
            " +---------++------+--------------+------+---------+----------+");
    System.out.println(String.format(" | CLK (13)|| #%02d  | %s | CLK  | GPIO_%02d | %02d       |",
        PinUtil.findByPin(clk).pinNumber(),
        StringUtils.rpad(PinUtil.findByPin(clk).pinName(), 12, " "),
        PinUtil.findByPin(clk).gpio(),
        PinUtil.findByPin(clk).wiringPi()));
    System.out.println(String.format(" | Din (11)|| #%02d  | %s | MOSI | GPIO_%02d | %02d       |",
        PinUtil.findByPin(mosi).pinNumber(),
        StringUtils.rpad(PinUtil.findByPin(mosi).pinName(), 12, " "),
        PinUtil.findByPin(mosi).gpio(),
        PinUtil.findByPin(mosi).wiringPi()));
    System.out.println(String.format(" | Dout(12)|| #%02d  | %s | MISO | GPIO_%02d | %02d       |",
        PinUtil.findByPin(miso).pinNumber(),
        StringUtils.rpad(PinUtil.findByPin(miso).pinName(), 12, " "),
        PinUtil.findByPin(miso).gpio(),
        PinUtil.findByPin(miso).wiringPi()));
    System.out.println(String.format(" | CS  (10)|| #%02d  | %s | CS   | GPIO_%02d | %02d       |",
        PinUtil.findByPin(cs).pinNumber(),
        StringUtils.rpad(PinUtil.findByPin(cs).pinName(), 12, " "),
        PinUtil.findByPin(cs).gpio(),
        PinUtil.findByPin(cs).wiringPi()));
    System.out.println(" +---------++------+--------------+-----+----------+----------+");
    System.out.println("Raspberry Pi is the Master, MCP3008 is the Slave:");
    System.out.println("- Dout on the MCP3008 goes to MISO on the RPi");
    System.out.println("- Din on the MCP3008 goes to MOSI on the RPi");
    System.out.println("Pins on the MCP3008 are numbered from 1 to 16, beginning top left, counter-clockwise.");
    System.out.println("       +--------+ ");
    System.out.println(String.format("%s CH0 -+  1  16 +- Vdd ",  (adcChannel == 0 ? "*" : " ")));
    System.out.println(String.format("%s CH1 -+  2  15 +- Vref ", (adcChannel == 1 ? "*" : " ")));
    System.out.println(String.format("%s CH2 -+  3  14 +- aGnd ", (adcChannel == 2 ? "*" : " ")));
    System.out.println(String.format("%s CH3 -+  4  13 +- CLK ",  (adcChannel == 3 ? "*" : " ")));
    System.out.println(String.format("%s CH4 -+  5  12 +- Dout ", (adcChannel == 4 ? "*" : " ")));
    System.out.println(String.format("%s CH5 -+  6  11 +- Din ",  (adcChannel == 5 ? "*" : " ")));
    System.out.println(String.format("%s CH6 -+  7  10 +- CS ",   (adcChannel == 6 ? "*" : " ")));
    System.out.println(String.format("%s CH7 -+  8   9 +- dGnd ", (adcChannel == 7 ? "*" : " ")));
    System.out.println("       +--------+ ");

    // Compose mapping for PinUtil
    String[] map = new String[4];
    map[0] = String.valueOf(PinUtil.findByPin(clk).pinNumber()) + ":" + "CLK";
    map[1] = String.valueOf(PinUtil.findByPin(miso).pinNumber()) + ":" + "Dout/MISO";
    map[2] = String.valueOf(PinUtil.findByPin(mosi).pinNumber()) + ":" + "Din/MOSI";
    map[3] = String.valueOf(PinUtil.findByPin(cs).pinNumber()) + ":" + "CS";

    PinUtil.print(map);

Reading MCP3008 on channel 0
 Wiring of the MCP3008-SPI (without power supply):
 +---------++-------------------------------------------------+
 | MCP3008 || Raspberry Pi                                    |
 +---------++------+--------------+------+---------+----------+
 |         || Pin# | Name         | Role | GPIO    | wiringPI |
 |         ||      |              |      | /BCM    | /PI4J    |
 +---------++------+--------------+------+---------+----------+
 | CLK (13)|| #12  | PCM_CLK/PWM0 | CLK  | GPIO_18 | 01       |
 | Din (11)|| #18  | GPIO_5       | MOSI | GPIO_24 | 05       |
 | Dout(12)|| #16  | GPIO_4       | MISO | GPIO_23 | 04       |
 | CS  (10)|| #22  | GPIO_6       | CS   | GPIO_25 | 06       |
 +---------++------+--------------+-----+----------+----------+
Raspberry Pi is the Master, MCP3008 is the Slave:
- Dout on the MCP3008 goes to MISO on the RPi
- Din on the MCP3008 goes to MOSI on the RPi
Pins on the MCP3008 are numbered from 1 to 16, beginning top left, counter-

All seems to be good, let us initialize the `MCP3008`:

In [6]:
try {
    MCP3008Reader.initMCP3008(miso, mosi, clk, cs);
} catch (Throwable error) {
    System.err.println("Not on a RPi?");
}

Not on a RPi?


The 3 values below are used to display the values on change only:

In [7]:
int lastRead = 0;
int tolerance = 5;
boolean first = true;

Here is a loop, used to read the values returned by the `MCP3008`. 

When the program is looping, tune the potentiometer to make the values change:

In [8]:
// Reading loop
for (int i=0; i<10; i++) {
  // System.out.println("Reading channel " + adcChannel);
  boolean simulating = false;
  int adc = 0;
  try { 
      adc = MCP3008Reader.readMCP3008(adcChannel);
  } catch (NullPointerException npe) {
      simulating = true;
      adc = (int)Math.round(1_024D * Math.random());
  } catch (Throwable error) {
      error.printStackTrace();
  }
  // System.out.println(String.format("From ch %d: %d", adcChannel, adc));
  int postAdjust = Math.abs(adc - lastRead);
  if (first || postAdjust > tolerance) {
    int volume = (int) (adc / 10.23); // [0, 1023] ~ [0x0000, 0x03FF] ~ [0&0, 0&1111111111]
    if (DEBUG) {
      System.out.println("readAdc" + (simulating ? " (simulating)" : "") + ":" + Integer.toString(adc) +
          " (0x" + StringUtils.lpad(Integer.toString(adc, 16).toUpperCase(), 2, "0") +
          ", 0&" + StringUtils.lpad(Integer.toString(adc, 2), 8, "0") + ")");
    }
    System.out.println(String.format("Volume: %s%% (%04d) => %.03f V",
        StringUtils.lpad(String.format("%d", volume), 3, " "),
        adc,
        (3.3 * (adc / 1023.0))));  // Volts
    lastRead = adc;
    first = false;
  }
  try {
    synchronized (Thread.currentThread()) {
      Thread.currentThread().wait(1_000L); // in milliseconds
    }
  } catch (InterruptedException ie) {
    Thread.currentThread().interrupt();
  }
}

readAdc (simulating):49 (0x31, 0&00110001)
Volume:   4% (0049) => 0.158 V
readAdc (simulating):971 (0x3CB, 0&1111001011)
Volume:  94% (0971) => 3.132 V
readAdc (simulating):126 (0x7E, 0&01111110)
Volume:  12% (0126) => 0.406 V
readAdc (simulating):500 (0x1F4, 0&111110100)
Volume:  48% (0500) => 1.613 V
readAdc (simulating):109 (0x6D, 0&01101101)
Volume:  10% (0109) => 0.352 V
readAdc (simulating):339 (0x153, 0&101010011)
Volume:  33% (0339) => 1.094 V
readAdc (simulating):903 (0x387, 0&1110000111)
Volume:  88% (0903) => 2.913 V
readAdc (simulating):393 (0x189, 0&110001001)
Volume:  38% (0393) => 1.268 V
readAdc (simulating):163 (0xA3, 0&10100011)
Volume:  15% (0163) => 0.526 V
readAdc (simulating):89 (0x59, 0&01011001)
Volume:   8% (0089) => 0.287 V


And finally, we free the resources:

In [9]:
System.out.println("Bye, freeing resources.");
try {
    MCP3008Reader.shutdownMCP3008();
} catch (NullPointerException npe) {
    System.err.println("Oops");
}

Bye, freeing resources.


Oops
